In [1]:
# import numpy as np
import cvxpy as cp
import networkx as nx
import autograd.numpy as np
from autograd import grad

import pymanopt
import pymanopt.manifolds
import pymanopt.solvers

from numpy.random import default_rng
from complex_elliptope import ComplexElliptope

In [2]:
def decompose_psd(X):
    n = X.shape[0]
    eigen_values, eigen_vectors = np.linalg.eigh(X)
    non_neg_eig_idx = [i for i in range(n) if eigen_values[i] >= 0]
    return eigen_vectors[:, non_neg_eig_idx] @ np.diag(np.sqrt(eigen_values[non_neg_eig_idx]))

In [3]:
def normalize_rows(X):
    # normalize the rows of matrix X
    return X / np.linalg.norm(X, axis=1)[:, np.newaxis]

In [4]:
def complex_hyperplane_rounding(Y, cost, iter=100):
    min_cost = 0
    best_x = None
    d = Y.shape[1]
    rng = default_rng()
    for i in range(iter):
        r = rng.random((d,1)) + rng.random((d,1)) * 1j
        r = r / np.linalg.norm(r)
        x = Y @ r
        x = x / np.linalg.norm(x)
        cost_val = cost(x)
        if cost_val < min_cost:
            min_cost = cost_val
            best_x = x
    return best_x, min_cost

In [5]:
def get_rand_complex(m, n, max_val=5):
    return np.round(np.random.rand(m, n) * max_val) + np.round(np.random.rand(m, n) * max_val) * 1j

# Phase recovery

In [6]:
n = 50  # number of observations
p = 20  # dimension of x
max_val = 1e2

In [7]:
rng = default_rng()
A = rng.random((n,p)) * max_val + rng.random((n,p)) * max_val * 1j
assert np.linalg.matrix_rank(A, tol=1e-6) >= p  # A must be injective; if the rows of A are linearly independent, AA+ = I
b = rng.random(n) * max_val
M = np.diag(b) @ (np.identity(n) - A @ np.linalg.pinv(A)) @ np.diag(b)

\begin{equation*}
\begin{aligned}
\min \quad & u^{*}Mu \\
\textrm{s.t.} \quad & |u_{i}| = 1, \qquad i=1,...,n
\end{aligned}
\end{equation*}

In [8]:
# u = cp.Variable(n, complex=True)
# qp_constraints = [ u[i].value.H * u[i] == 1 for i in range(n) ]
# prob = cp.Problem(cp.Minimize(cp.quad_form(u, M)), qp_constraints)
# prob.solve()

\begin{equation*}
\begin{aligned}
\min \quad & M \cdot U \\
\textrm{s.t.} \quad & U_{ii} = 1, \qquad i=1,...,n \\
& U \succeq 0
\end{aligned}
\end{equation*}

### solve SDP with cvxpy

In [9]:
U = cp.Variable((n,n), hermitian=True)
# U = cp.Variable((n,n), symmetric=True)  # for testing

In [10]:
sdp_constraints = [U >> 0]
sdp_constraints += [U[i][i] == 1 for i in range(n)]

In [11]:
prob = cp.Problem(cp.Minimize(cp.real(cp.trace(M @ U))), sdp_constraints)
prob.solve()

print("The optimal value is", prob.value)
print("A solution U is", U.value)

The optimal value is 133.17786652587796
A solution U is [[ 1.        +0.j         -0.00131718+0.2186223j   0.4020449 +0.59538451j
  ... -0.17726416+0.44345426j  0.08777712-0.13906409j
  -0.36502408-0.09934923j]
 [-0.00131718-0.2186223j   1.        +0.j         -0.08230402+0.52451177j
  ...  0.9088733 +0.19196539j  0.80070221-0.26189672j
  -0.86710893-0.07977449j]
 [ 0.4020449 -0.59538451j -0.08230402-0.52451177j  1.        +0.j
  ...  0.10451819-0.22552914j -0.3458085 -0.66616775j
  -0.17721505+0.76617689j]
 ...
 [-0.17726416-0.44345426j  0.9088733 -0.19196539j  0.10451819+0.22552914j
  ...  1.        +0.j          0.49794428-0.39248888j
  -0.70752326+0.24315146j]
 [ 0.08777712+0.13906409j  0.80070221+0.26189672j -0.3458085 +0.66616775j
  ...  0.49794428+0.39248888j  1.        +0.j
  -0.79002608-0.51001191j]
 [-0.36502408+0.09934923j -0.86710893+0.07977449j -0.17721505-0.76617689j
  ... -0.70752326-0.24315146j -0.79002608+0.51001191j
   1.        +0.j        ]]


In [12]:
opt_rank = np.linalg.matrix_rank(U.value, 1e-9)
print(opt_rank)

4


### Complex elliptope gradient

In [13]:
manifold = ComplexElliptope(n, opt_rank)

@pymanopt.function.Autograd(manifold)
def manifold_cost(Y):
    return np.real(np.trace(M @ Y @ np.conj(Y).T))

problem = pymanopt.Problem(manifold=manifold, cost=manifold_cost)
solver = pymanopt.solvers.SteepestDescent(minstepsize=1e-6)
solution = solver.solve(problem)

Optimizing...
Iteration    Cost                       Gradient norm     
---------    -----------------------    --------------    
   1         +9.6775038073735224e+04    3.24166703e+04    
Terminated - min stepsize reached after 1 iterations, 0.00 seconds.



### Hyperplane rounding

In [15]:
qp_cost = lambda u: cp.real(cp.quad_form(u, M)).value

In [16]:
complex_hyperplane_rounding(decompose_psd(U.value), qp_cost)

(array([[-9.19539020e-01+0.39299872j],
        [ 2.21702498e-01+0.97511435j],
        [ 9.60493600e-01+0.27830207j],
        [-3.37973525e-01-0.94115562j],
        [-9.89122506e-01+0.14709408j],
        [ 3.28219242e-01-0.94460157j],
        [-5.91089430e-01-0.80660603j],
        [ 9.99919288e-01-0.01270502j],
        [ 9.92054973e-01-0.12580513j],
        [ 8.83485806e-01+0.46845793j],
        [ 5.52464231e-01+0.83353661j],
        [-9.05950052e-01+0.42338458j],
        [ 4.42164275e-01-0.89693409j],
        [-8.55832330e-01-0.51725335j],
        [-9.62069382e-01-0.27280488j],
        [-4.64072713e-01-0.88579711j],
        [-6.77465075e-01-0.73555494j],
        [-7.33705805e-01-0.67946729j],
        [-6.39236088e-01+0.76901055j],
        [-7.70955240e-01+0.63688933j],
        [-9.10744428e-01-0.41297044j],
        [-9.13171459e-01+0.40757562j],
        [-9.37212121e-01+0.34875986j],
        [ 4.34124065e-01-0.90085309j],
        [-7.43373148e-01+0.66887694j],
        [-8.74930595e-01+